In [ ]:
PROJECT_ROOT = None

# Compute Transmission × Historical Fire Exposure (Length-Weighted)

This notebook computes screening-level exposure of publicly identifiable ≥115 kV transmission lines to historical wildfire occurrence by intersecting transmission geometry with CAL FIRE FRAP fire perimeters. All spatial computation is performed in PostGIS and cached for reuse. Length is measured in EPSG:3310 (California Albers) and reported in kilometers.

## Inputs (PostGIS)
- `tx_lines_3310` (cached in Notebook 04; EPSG:3310)
- `fire_perimeters` (canonical perimeters; EPSG:4326)

## Outputs (PostGIS)
- `fire_perimeters_3310` (perimeters transformed to EPSG:3310)
- `tx_overlap_any_fire_by_line` (line_id, overlap_km)

## Outputs (Files)
- `outputs/tables/tx_overlap_any_fire_total.csv`
- `outputs/tables/tx_overlap_any_fire_by_line.csv`

### Imports

In [1]:
import os
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

### Define Paths

In [2]:
CWD = Path.cwd()
ROOT = CWD.parent if CWD.name.lower() == "notebooks" else CWD

OUTPUT_TABLES = ROOT / "outputs" / "tables"
OUTPUT_TABLES.mkdir(parents=True, exist_ok=True)

load_dotenv(ROOT / ".env")
print("ROOT:", ROOT)
print("OUTPUT_TABLES:", OUTPUT_TABLES)

ROOT: C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure
OUTPUT_TABLES: C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure\outputs\tables


## Database Connection

In [3]:
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "wildfire_grid")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

if not DB_USER or not DB_PASSWORD:
    raise ValueError("Missing DB_USER or DB_PASSWORD in environment (.env).")

db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(db_url)

with engine.begin() as conn:
    conn.execute(text("SELECT 1;"))

print("DB:", DB_NAME, "@", DB_HOST, DB_PORT)

DB: wildfire_grid @ localhost 5432


## Pre-Checks

In [4]:
check_sql = """
SELECT
  to_regclass('public.tx_lines_3310') AS has_tx_lines_3310,
  (SELECT COUNT(*) FROM fire_perimeters) AS n_fire_perimeters;
"""
pd.read_sql(check_sql, engine)

,has_tx_lines_3310,n_fire_perimeters
0,tx_lines_3310,9474


## Cache fire_perimeters_3310

In [5]:
sql_cache_fire_3310 = """
DROP TABLE IF EXISTS fire_perimeters_3310;

CREATE TABLE fire_perimeters_3310 AS
SELECT
  fire_year,
  fire_name,
  source,
  ST_Transform(ST_MakeValid(geom), 3310) AS geom
FROM fire_perimeters
WHERE fire_year IS NOT NULL;

CREATE INDEX IF NOT EXISTS idx_fire_perimeters_3310_geom_gist ON fire_perimeters_3310 USING GIST (geom);
CREATE INDEX IF NOT EXISTS idx_fire_perimeters_3310_year ON fire_perimeters_3310 (fire_year);
ANALYZE fire_perimeters_3310;
"""

with engine.begin() as conn:
    conn.execute(text(sql_cache_fire_3310))

print("Created: fire_perimeters_3310")

Created: fire_perimeters_3310


## Compute Overlap: Transmission Lines Intersecting ANY Historical Fire Perimeter

In [8]:
sql_overlap_any_fire = """
DROP TABLE IF EXISTS tx_overlap_any_fire_by_line;

CREATE TABLE tx_overlap_any_fire_by_line AS
WITH fire_union AS (
  SELECT ST_UnaryUnion(ST_Collect(geom)) AS geom
  FROM fire_perimeters_3310
),
candidates AS (
  SELECT
    t.line_id,
    ST_Intersection(t.geom, f.geom) AS inter_geom
  FROM tx_lines_3310 t
  CROSS JOIN fire_union f
  WHERE ST_Intersects(t.geom, f.geom)
),
lines_only AS (
  SELECT
    line_id,
    ST_CollectionExtract(inter_geom, 2) AS inter_line
  FROM candidates
)
SELECT
  line_id,
  (ST_Length(inter_line) / 1000.0) AS overlap_km
FROM lines_only
WHERE inter_line IS NOT NULL
  AND NOT ST_IsEmpty(inter_line);

CREATE INDEX IF NOT EXISTS idx_tx_overlap_any_fire_line
  ON tx_overlap_any_fire_by_line (line_id);

ANALYZE tx_overlap_any_fire_by_line;
"""

with engine.begin() as conn:
    conn.execute(text(sql_overlap_any_fire))

print("Created: tx_overlap_any_fire_by_line")

Created: tx_overlap_any_fire_by_line


### Total Overlap Length Statewide

In [9]:
df_total = pd.read_sql(
    """
    SELECT
      SUM(overlap_km) AS total_overlap_km,
      COUNT(*) AS n_lines_with_overlap
    FROM tx_overlap_any_fire_by_line;
    """,
    engine
)
df_total

,total_overlap_km,n_lines_with_overlap
0,156.933994,133


## Export CSVs

In [10]:
df_total.to_csv(OUTPUT_TABLES / "tx_overlap_any_fire_total.csv", index=False)

df_by_line = pd.read_sql(
    """
    SELECT line_id, overlap_km
    FROM tx_overlap_any_fire_by_line
    ORDER BY overlap_km DESC;
    """,
    engine
)
df_by_line.to_csv(OUTPUT_TABLES / "tx_overlap_any_fire_by_line.csv", index=False)

print("Wrote:")
print("-", OUTPUT_TABLES / "tx_overlap_any_fire_total.csv")
print("-", OUTPUT_TABLES / "tx_overlap_any_fire_by_line.csv")

Wrote:
- C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure\outputs\tables\tx_overlap_any_fire_total.csv
- C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure\outputs\tables\tx_overlap_any_fire_by_line.csv
